In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile,assemble
from qiskit import Aer, execute
from qiskit.extensions import Initialize
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import partial_trace, Statevector, random_statevector, Operator
from qiskit_textbook.tools import simon_oracle
import numpy as np
import math
import matplotlib.pyplot as plt


from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel

In [ ]:
backend = Aer.get_backend("qasm_simulator")
job = execute(qc, backend,shots=1000)
result = job.result()
plot_histogram(result.get_counts())

In [ ]:
def create_bell_state(circuit, q1, q2):
    circuit.h(q1)
    circuit.cnot(q1, q2)

def create_bell_state(circuit, register):
    circuit.h(register[0])
    circuit.cnot(register[0], register[1])

def teleport_gates(circuit, alice_qbit, alice_ent):
    circuit.cnot(alice_qbit, alice_ent)
    circuit.h(alice_qbit)


def swap(circuit, q1, q2):
    circuit.cnot(q1, q2)
    circuit.cnot(q2,q1)
    circuit.cnot(q1, q2)

# Data Encoding

## Basis state

Feature binarie. Dati i campioni assegnamo equo amplitude e nulla alle altre. Dato che 

$
\sum_{i=1}^{2^N} \alpha_i^2 = 1
$

la somma delle amplitude per ogni stato deve essere uguale ad uno, e vogliamo assegnare equa amplitude (solo per i dati presenti), questa sarà (con N numero campioni):

$
1 = \sum_{i=1}^{N} \alpha^2 = N\alpha^2  \rightarrow \alpha = \frac{1}{\sqrt{N}}
$

In [ ]:
N_FEATURES = 3
DATA = ['101', '111', '000']

state_preparation = [0] * (2**N_FEATURES)
for e in DATA:
    state_preparation[int(e, 2)] = 1/math.sqrt(len(DATA))

print(state_preparation)

qc = QuantumCircuit(N_FEATURES, N_FEATURES)
qc.initialize(state_preparation, range(N_FEATURES))


qc.decompose().draw(output="mpl")

## Amplitude

Per rappresentare un intero dataset concateno i dati di ogni elemento. Per un dataset in $\mathbb{R}^{n\times n}$ avrò quindi bisogno di   


$
N_{qbit} = n^2 log_2(n^2)
$


Dato quindi un dataset $X = (\alpha_1, \dots , \alpha_n)$ necessario rinormalizzare i dati in modo tale che, data una costante di normalizzazione $k$:

$
\sum_{i=1}^{n} |(k\alpha_i)^2| = 1
$
si ha quindi

$
k = \sqrt{\frac{1} {\sum_{i=1}^{n} |\alpha_i^2|}}
$

In [ ]:
DATA = np.array([[5,2,3],[1,3,1]])
N_QBIT = math.ceil(math.log2(DATA.size))
print(N_QBIT)
value =  math.sqrt(1/(np.sum(DATA.flatten()**2)))

state_preparation = np.append(DATA.flatten() * value, [0] * (2**N_QBIT - DATA.size))

qc = QuantumCircuit(N_QBIT, N_QBIT)
qc.initialize(state_preparation, range(N_QBIT))

qc.decompose().draw(output="mpl")


# Classification